In [1]:
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [2]:
#Load data
mnist = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train.shape, y_train.shape

170500096/170498071 [==============================] - 2s 0us/step


((50000, 32, 32, 3), (50000, 1))

In [19]:
IMG_SHAPE = (32, 32, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

/tensorflow-2.1.0/python3.6/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [20]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 33, 33, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [21]:
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  155


In [23]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  #tf.keras.layers.Dense(512, activation='relu'),
  #tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])

model_name = "cfar10_pretrained_model"
model.save(model_name)
model.summary()

INFO:tensorflow:Assets written to: cfar10_pretrained_model/assets
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 1, 1, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                12810     
Total params: 2,270,794
Trainable params: 1,875,402
Non-trainable params: 395,392
_________________________________________________________________


In [0]:
lr_step = tf.Variable(0, trainable=False)
lr_boundaries = [150, 250]
lr_values = [1e-2, 1e-3, 1e-4]
learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(lr_boundaries, lr_values)
# Later, whenever we perform an optimization step, we pass in the step.
lr_schedule = learning_rate_fn(lr_step)

In [0]:
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy', #'categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
filepath = 'cifar_pretrained_model.h5'

callback = [#tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
            tf.keras.callbacks.ModelCheckpoint(filepath=filepath, save_best_only=True, monitor='val_loss', verbose=1)]

model.fit(x_train, y_train, epochs=10, validation_split=0.2, callbacks=callback)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
39968/40000 [============================>.] - ETA: 0s - loss: 2.3260 - accuracy: 0.3928
Epoch 00001: val_loss improved from inf to 2.46783, saving model to cifar_pretrained_model.h5
40000/40000 [==============================] - 19s 477us/sample - loss: 2.3256 - accuracy: 0.3928 - val_loss: 2.4678 - val_accuracy: 0.1381
Epoch 2/10
39872/40000 [============================>.] - ETA: 0s - loss: 1.7578 - accuracy: 0.4841
Epoch 00002: val_loss improved from 2.46783 to 2.02832, saving model to cifar_pretrained_model.h5
40000/40000 [==============================] - 16s 404us/sample - loss: 1.7567 - accuracy: 0.4843 - val_loss: 2.0283 - val_accuracy: 0.3827
Epoch 3/10
39936/40000 [============================>.] - ETA: 0s - loss: 1.4425 - accuracy: 0.5422
Epoch 00003: val_loss did not improve from 2.02832
40000/40000 [==============================] - 16s 401us/sample - loss: 1.4424 - accuracy: 0.5421 - val_loss: 2.1781 - val_accu

In [0]:
model = tf.keras.models.load_model(filepath)

In [28]:
model.evaluate(x_train,  y_train, verbose=2)

50000/50000 - 12s - loss: 1.5674 - accuracy: 0.4515


[1.5674174377441406, 0.45148]

In [29]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 2s - loss: 1.5817 - accuracy: 0.4524


[1.5817406232833862, 0.4524]